In [4]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="bert-base-cased",
                  local_dir="../model/bert-base-cased")

config.json: 100%|██████████| 570/570 [00:00<00:00, 322kB/s]

README.md: 100%|██████████| 8.98k/8.98k [00:00<00:00, 6.49MB/s]

Fetching 10 files:  10%|█         | 1/10 [00:01<00:12,  1.38s/it]
tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 24.3kB/s]















tokenizer.json: 100%|██████████| 436k/436k [00:00<00:00, 551kB/s]

vocab.txt: 100%|██████████| 213k/213k [00:00<00:00, 361kB/s]
























































































































































































































































































































































































































































































































































































flax_model.msgpack: 10

'/Users/huangxinzhe/code/llm_fine_tuning/hugging_face_transformers/fine_tuning/fine_tune_quickstart/model'

In [1]:
import datasets

/Users/huangxinzhe/opt/anaconda3/envs/llm9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full",
                       cache_dir='/Users/huangxinzhe/code/llm_fine_tuning/hugging_face_transformers/fine_tuning/data')

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "/Users/huangxinzhe/code/llm_fine_tuning/hugging_face_transformers/fine_tuning/fine_tune_quickstart/model/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map: 100%|██████████| 50000/50000 [00:04<00:00, 11136.75 examples/s]


In [8]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [9]:
def show_random_elements(dataset, num_examples=10):
    # assert是断言，如果后面的表达式为真，则继续执行，否则抛出AssertionError
    # 判断随机选取的样本数是否大于数据集的样本数
    assert num_examples <= len(
        dataset), "Can't pick more elements than there are in the dataset."
    # 随机选取num_examples个样本
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,3 stars,"This place is Uber expensive. I take my wife here once a year with some high roller friends that get everything on comp. Otherwise I wold never step foot in here. The atmosphere is very Bling. A unique gas fire place compliments the room amongst crystal chandeliers. Live Piano Music can be heard in the Bar at Happy Hours into dinner time. The food well. I never could justify a 50 dollar steak on a plate. I'm not blown away by steak house cuisine. Never was. But I guess you can say its the best money can buy. Our host orders a bunch of things up an down the menu and the tab can get into the thousands. I was never impressed by any of the dishes. Steak Plate, portions small, Drinks are on the other hand real good. The Wine List is Pro casino style. They got all the names and almost all the bottles that are currantly desired.","[101, 1188, 1282, 1110, 158, 3169, 5865, 119, 146, 1321, 1139, 1676, 1303, 1517, 170, 1214, 1114, 1199, 1344, 14167, 2053, 1115, 1243, 1917, 1113, 3254, 1643, 119, 15853, 146, 192, 11015, 1309, 2585, 2555, 1107, 1303, 119, 1109, 6814, 1110, 1304, 139, 1979, 119, 138, 3527, 3245, 1783, 1282, 21215, 1116, 1103, 1395, 5690, 8626, 22572, 5709, 26041, 1116, 119, 3374, 8802, 1953, 1169, 1129, 1767, 1107, 1103, 6523, 1120, 8325, 13819, 1154, 4014, 1159, 119, 1109, 2094, 1218, 119, 146, 1309, 1180, 17422, 170, 1851, 8876, 26704, 1113, 170, 4885, 119, 146, 112, 182, 1136, 10816, 1283, 1118, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(
    seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(
    seed=42).select(range(1000))